## Federal registry API pull


https://www.federalregister.gov/developers/api/v1  
https://www.federalregister.gov/agencies

In [1]:
import json
import requests
from bs4 import BeautifulSoup
import json
import re
import urllib
import pandas as pd
import numpy as np
from pandas.io.json import json_normalize

In [2]:
# Pull from all agencies
number='2000'
page='2'
url='https://www.federalregister.gov/api/v1/documents.json?order=relevance&page='+page+'&per_page='+number

In [3]:
# Specify a single agency
# number='20'
# string='defense-department'
# url = "https://www.federalregister.gov/api/v1/documents.json?per_page="+number+"&order=relevance&conditions%5Bagencies%5D%5B%5D=" + string

In [4]:
# get request
response = requests.get(url).json()
len(response)

5

## Parse the response of the get request

In [5]:
response.keys()

dict_keys(['count', 'description', 'total_pages', 'previous_page_url', 'results'])

In [6]:
list(response)

['count', 'description', 'total_pages', 'previous_page_url', 'results']

In [8]:
print(response['count'])
print(response['description'])
print(response['total_pages'])
print(response['next_page_url'])

778626
All Documents
5


### Look at the results

In [9]:
response['results'][0]

{'abstract': None,
 'agencies': [{'id': 188,
   'json_url': 'https://www.federalregister.gov/api/v1/agencies/188.json',
   'name': 'Federal Reserve System',
   'parent_id': None,
   'raw_name': 'FEDERAL RESERVE SYSTEM',
   'slug': 'federal-reserve-system',
   'url': 'https://www.federalregister.gov/agencies/federal-reserve-system'}],
 'document_number': '2018-02635',
 'excerpts': None,
 'html_url': 'https://www.federalregister.gov/documents/2018/02/09/2018-02635/formations-of-acquisitions-by-and-mergers-of-bank-holding-companies',
 'pdf_url': 'https://www.gpo.gov/fdsys/pkg/FR-2018-02-09/pdf/2018-02635.pdf',
 'public_inspection_pdf_url': 'https://s3.amazonaws.com/public-inspection.federalregister.gov/2018-02635.pdf?1518097574',
 'publication_date': '2018-02-09',
 'title': 'Formations of, Acquisitions by, and Mergers of Bank Holding Companies',
 'type': 'Notice'}

In [10]:
len(response['results'][0])

10

In [11]:
# Response has 10 keys
response['results'][0].keys()

dict_keys(['title', 'type', 'abstract', 'document_number', 'html_url', 'pdf_url', 'public_inspection_pdf_url', 'publication_date', 'agencies', 'excerpts'])

In [12]:
# One of those keys, agencies, is a dictionary with 7 additional keys
response['results'][0]['agencies'][0].keys()

dict_keys(['raw_name', 'name', 'id', 'url', 'json_url', 'parent_id', 'slug'])

## Flatten the file

In [13]:
# Flatten the dataset, WITHOUT creating duplicate rows when two agencies are listed.
df = json_normalize(response['results'])

In [14]:
# Flatten the "agencies" dictionary column into a new dataset
df2 = pd.DataFrame(df['agencies'].values.tolist(), index=df.index)

In [15]:
# Extract the name of the sub-agency as a separate column
def extractor (col):
    try:
        return(col['name'])
    except:
        pass
df2['subagency']=df2[1].apply(extractor)

In [16]:
# Flatten the first agency and its metadata into a separate dataset
df3=pd.DataFrame(df2[0].values.tolist(), index=df2.index)

In [17]:
# Join the first-agency dataset with the subagency column
df4=df3.join(df2, how='outer').drop([0,1], axis=1)

In [18]:
# Join that dataset into the original dataset of Registry entries
df5=df.join(df4,how='outer').drop('agencies', axis=1)

In [19]:
print(df5.shape)
print(df5.columns)

(2000, 31)
Index([                 'abstract',           'document_number',
                        'excerpts',                  'html_url',
                         'pdf_url', 'public_inspection_pdf_url',
                'publication_date',                     'title',
                            'type',                        'id',
                        'json_url',                      'name',
                       'parent_id',                  'raw_name',
                            'slug',                       'url',
                                 2,                           3,
                                 4,                           5,
                                 6,                           7,
                                 8,                           9,
                                10,                          11,
                                12,                          13,
                                14,                          15,
              

In [20]:
df5.head(2)

abstract document_number  \
0                                               None      2018-02635   
1  Under the provisions of the Paperwork Reductio...      2018-02587   

                                            excerpts  \
0                                               None   
1  Under the provisions of the Paperwork Reductio...   

                                            html_url  \
0  https://www.federalregister.gov/documents/2018...   
1  https://www.federalregister.gov/documents/2018...   

                                             pdf_url  \
0  https://www.gpo.gov/fdsys/pkg/FR-2018-02-09/pd...   
1  https://www.gpo.gov/fdsys/pkg/FR-2018-02-09/pd...   

                           public_inspection_pdf_url publication_date  \
0  https://s3.amazonaws.com/public-inspection.fed...       2018-02-09   
1  https://s3.amazonaws.com/public-inspection.fed...       2018-02-09   

                                               title    type   id  \
0  Formations of, Acquisitions by, and Mergers of...  Notice  188   
1  Submission for OMB Review; Prohibition of Acqu...  Notice  103   

                ...                    7     8     9    10    11    12    13  \
0               ...                 None  None  None  None  None  None  None   
1               ...                 None  None  None  None  None  None  None   

     14    15                        subagency  
0  None  None                             None  
1  None  None  General Services Administration  

[2 rows x 31 columns]

In [21]:
df5['subagency'].head(5)

0                                          None
1               General Services Administration
2    Centers for Disease Control and Prevention
3    Centers for Disease Control and Prevention
4      Centers for Medicare & Medicaid Services
Name: subagency, dtype: object

In [22]:
df5.rename(columns={'name':'agency'}, inplace=True)
df5.rename(columns={'id':'agency_id'}, inplace=True)

In [23]:
df5.columns

Index([                 'abstract',           'document_number',
                        'excerpts',                  'html_url',
                         'pdf_url', 'public_inspection_pdf_url',
                'publication_date',                     'title',
                            'type',                 'agency_id',
                        'json_url',                    'agency',
                       'parent_id',                  'raw_name',
                            'slug',                       'url',
                                 2,                           3,
                                 4,                           5,
                                 6,                           7,
                                 8,                           9,
                                10,                          11,
                                12,                          13,
                                14,                          15,
                       's

In [24]:
df5=df5[['document_number', 'publication_date', 'agency_id', 'agency', 'subagency', 
        'title', 'abstract', 'type', 'excerpts', 'raw_name', 'slug', 'url', 'html_url', 
        'pdf_url', 'public_inspection_pdf_url',  'json_url', 'parent_id']]

## Export to CSV

In [25]:
from datetime import date
today = str(date.today())
today

'2018-05-22'

In [26]:
import datetime
def _getToday():
    return datetime.datetime.now().strftime("%y-%m-%d-%H-%M")
outpath = r'C:/Users/alasseter/Documents/Projects/CTxE/Federal Register/data/' 
filename = "%s_%s.%s" % ("fedreg", _getToday() ,"csv")
df5.to_csv(outpath + filename, index=False)
print (outpath + filename)

C:/Users/alasseter/Documents/Projects/CTxE/Federal Register/data/fedreg_18-05-22-14-53.csv
